In [1]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, random_split

In [ ]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)), 
        transforms.RandomRotation(degrees=15), 
        transforms.ColorJitter(), 
        transforms.RandomHorizontalFlip(), 
        transforms.CenterCrop(size=224),  # Image net standards 
        transforms.ToTensor(), 
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])  # Imagenet standards
])

dataset = ImageFolder(root='./101_ObjectCategories', transform=transform)
train_size = int(0.5 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [ ]:
import torch.nn as nn
import torchvision.models as models

# Load pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)

# Freeze the internal layers
for param in vgg16.parameters():
    param.requires_grad = False

# Replace the classifier
vgg16.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Linear(512 * 7 * 7, 100),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Dropout(0.2),
    nn.Linear(100, 102)  # Adjust output size to match the number of classes
)

# Move model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vgg16.to(device)

In [ ]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.classifier.parameters(), lr=0.001)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    vgg16.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

    # Validation loop (optional)
    vgg16.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = vgg16(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Validation Loss: {val_loss/len(val_loader)}, Accuracy: {100 * correct / total}%")